In [31]:
!pip install transformers datasets accelerate evaluate

In [ ]:
import pandas as pd
url = 'https://github.com/e9t/nsmc/raw/master/ratings_train.txt'
df = pd.read_csv(url, sep='\t').dropna()

In [ ]:
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
from datasets import Dataset

In [ ]:
def gen():
  for row in df.itertuples():
    yield {'text': row.document, 'label': row.label}

In [ ]:
ds = Dataset.from_generator(gen)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(
"skt/kogpt2-base-v2",
bos_token='</s>',
eos_token='</s>',
unk_token='<unk>',
pad_token='<pad>',
mask_token='<mask>')

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

In [ ]:
tokenized_ds = ds.map(tokenize_function, batched=True)

Map:   0%|          | 0/149995 [00:00<?, ? examples/s]

In [ ]:
total_size = len(ds)

In [ ]:
train_size = int(total_size * 0.9)
train_ds = tokenized_ds.shuffle(seed=42).select(range(train_size))

In [ ]:
eval_ds = tokenized_ds.shuffle(seed=42).select(range(train_size, total_size))

In [ ]:
import torch
torch.cuda.is_available()

False

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(
"skt/kogpt2-base-v2", num_labels=2)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import evaluate
metric = evaluate.load("accuracy")

In [ ]:
import numpy as np
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
output_dir="test_trainer",
num_train_epochs=1,
evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
model=model,
args=training_args,
train_dataset=train_ds,
eval_dataset=eval_ds,
compute_metrics=compute_metrics,
)

In [ ]:
#trainer.train()

Epoch,Training Loss,Validation Loss


In [32]:
model_path = '/content/drive/Othercomputers/내 노트북/study/09_GenerativeAI/cache_dir/kogpt2-nsmc'
model.save_pretrained(model_path)

In [33]:
model = AutoModelForSequenceClassification.from_pretrained(
model_path, num_labels=2)

In [34]:
from transformers import pipeline

In [35]:
model.config.id2label = {0: 'NEGATIVE', 1: 'POSITIVE'}

In [36]:
sm = pipeline('text-classification', model=model,
tokenizer=tokenizer, device='cuda:0')

In [37]:
sm('이 영화 재밌다')

[{'label': 'POSITIVE', 'score': 0.9821693301200867}]

In [38]:
sm('돈 아깝다')

[{'label': 'NEGATIVE', 'score': 0.9971115589141846}]

In [39]:
sm('볼만한영화였지만 돈은 좀 아깝다')

[{'label': 'NEGATIVE', 'score': 0.9628980755805969}]